# V1

In [ ]:
# ==============================================================================
# PROJECT: ADVANCED INDOOR MICROBIAL RISK ASSESSMENT SYSTEM (8-WEEK PROJECT SCOPE)
# VERSION: 4.1 (Hyperopt Error Fix)
#
# KEY UPGRADES:
# 1. SCIENTIFIC RE-FRAMING: The model no longer predicts a non-existent "mould"
#    outcome. It now predicts a scientifically valid, multi-class "Environmental State"
#    based on ASHRAE/building science principles, serving as an early warning system.
# 2. HIERARCHICAL STATE DEFINITION: The target variable is now defined by a
#    prioritized set of rules (Sustained Dampness > Poor Ventilation > Elevated Humidity > Optimal).
# 3. ROBUST PIPELINE: The existing advanced features and ML pipeline are now
#    leveraged to predict these meaningful, real-world environmental states.
# ==============================================================================

# ------------------------------------------------------------------------------
# SECTION A: SETUP AND ENVIRONMENT
# ------------------------------------------------------------------------------
# ✅ Install compatible versions for stability
# !pip install --quiet numpy pandas scikit-learn imbalanced-learn xgboost shap hyperopt seaborn

# ✅ Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shap
import warnings

warnings.filterwarnings("ignore")

# ✅ Scikit-learn, Imblearn, XGBoost, Hyperopt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# ✅ Reproducibility
SEED = 42
np.random.seed(SEED)

class MicrobialRiskAssessor:
    """
    An advanced pipeline for assessing indoor environmental states conducive to
    microbial risk, acting as an early warning system.
    """
    def __init__(self, file_path, roll_window=24, rh_threshold=80.0):
        self.file_path = file_path
        self.roll_window = roll_window
        self.rh_threshold = rh_threshold # Humidity % for "wet" conditions
        self.df = None
        self.X = None
        self.y = None
        self.model = None
        self.scaler = None
        self.encoder = None
        self.feature_names = None
        print("✅ MicrobialRiskAssessor initialized.")

    def _load_and_clean_data(self):
        """Loads and cleans the initial dataset."""
        print("--> STAGE 1: Loading and Cleaning Data...")
        self.df = pd.read_csv(self.file_path)
        
        self.df.rename(columns=lambda c: 
                       c.replace('(?C)', '').replace('(%)', '').replace('(ppm)', '')
                       .replace('(?g/m?)', '').replace('(ppb)', '').replace('(lux)', '')
                       .strip().replace(' ', '_'), 
                       inplace=True)
        
        self.df['Timestamp'] = pd.to_datetime(self.df['Timestamp'], dayfirst=True)
        self.df.set_index('Timestamp', inplace=True)
        self.df.sort_index(inplace=True)
        self.df.ffill(inplace=True)
        print("    Data loaded and cleaned.")

    def _engineer_features_advanced(self):
        """Generates advanced features including volatility, lag, and anomaly scores."""
        print("--> STAGE 2: Advanced Feature Engineering...")
        # Volatility Features
        self.df['Hum_24_std'] = self.df['Humidity'].rolling(self.roll_window, min_periods=1).std()
        self.df['CO2_24_std'] = self.df['CO2'].rolling(self.roll_window, min_periods=1).std()

        # Time of Wetness (ToW) Calculation: Cumulative hours above RH threshold in the last 3 days
        is_wet = (self.df['Humidity'] > self.rh_threshold).astype(int)
        self.df['Time_of_Wetness_72h'] = is_wet.rolling(window=self.roll_window * 3, min_periods=1).sum()
        
        # Unsupervised Anomaly Detection (Isolation Forest)
        print("    Running Isolation Forest for anomaly detection...")
        numeric_cols = self.df.select_dtypes(include=np.number).columns
        if_model = IsolationForest(contamination=0.01, random_state=SEED)
        if_model.fit(self.df[numeric_cols])
        self.df['anomaly_score'] = if_model.decision_function(self.df[numeric_cols])
        
        self.df.ffill(inplace=True)
        print("    Advanced features created.")

    def _define_environmental_state(self):
        """
        Defines the target variable based on a hierarchy of scientifically-backed
        environmental states conducive to microbial growth.
        """
        print("--> STAGE 3: Defining Environmental State (Target Variable)...")

        def get_state(row):
            # Priority 1: Sustained Dampness (most critical)
            if row['Time_of_Wetness_72h'] > 4: # More than 4 hours of high RH in last 3 days
                return 'Sustained_Dampness'
            # Priority 2: Poor Ventilation (major contributing factor)
            elif row['CO2'] > 1000: # ASHRAE guideline for acceptable CO2
                return 'Poor_Ventilation'
            # Priority 3: Elevated Humidity (initial warning sign)
            elif row['Humidity'] > 60: # ASHRAE guideline for humidity
                return 'Elevated_Humidity'
            # Default State: Optimal Conditions
            else:
                return 'Optimal'

        self.df['Environmental_State'] = self.df.apply(get_state, axis=1)
        print("    Environmental states defined.")
        print("\nEnvironmental State Distribution:")
        print(self.df['Environmental_State'].value_counts())

    def _prepare_for_modeling(self):
        """Prepares final X, y dataframes, scales, and balances the data."""
        print("--> STAGE 4: Preparing Data for Final Model...")
        self.df.dropna(inplace=True)
        self.y = self.df['Environmental_State']
        self.X = self.df.drop(columns=['Environmental_State']) # Drop the target
        self.X = pd.get_dummies(self.X, columns=['Ventilation_Status'], drop_first=True)
        self.feature_names = self.X.columns.tolist()

        self.encoder = LabelEncoder()
        y_encoded = self.encoder.fit_transform(self.y)

        if len(self.encoder.classes_) < 2:
            raise ValueError("The target variable has less than 2 classes. Cannot proceed.")

        split_idx = int(len(self.X) * 0.8)
        X_train, self.X_test = self.X.iloc[:split_idx], self.X.iloc[split_idx:]
        y_train, self.y_test = y_encoded[:split_idx], y_encoded[split_idx:]

        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        self.X_test_scaled = self.scaler.transform(self.X_test)

        sm = SMOTE(random_state=SEED)
        self.X_train_bal, self.y_train_bal = sm.fit_resample(X_train_scaled, y_train)
        print(f"    Data prepared. Training set balanced with SMOTE: {self.X_train_bal.shape}")

    def _tune_and_train_model(self):
        """Uses Hyperopt to find the best XGBoost parameters and trains the final model."""
        print("--> STAGE 5: Hyperparameter Tuning and Training...")
        
        # --- FIX: Create a dedicated validation set for Hyperopt to avoid the __sklearn_tags__ error ---
        # This split is only for the optimization process.
        X_train_opt, X_val_opt, y_train_opt, y_val_opt = train_test_split(
            self.X_train_bal, self.y_train_bal, test_size=0.2, random_state=SEED
        )
        
        space = {
            'n_estimators': hp.quniform('n_estimators', 100, 800, 50),
            'max_depth': hp.quniform('max_depth', 4, 15, 1),
            'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
            'gamma': hp.uniform('gamma', 0, 0.5)
        }

        def objective(params):
            params['n_estimators'] = int(params['n_estimators'])
            params['max_depth'] = int(params['max_depth'])
            
            clf = XGBClassifier(**params, objective='multi:softmax', num_class=len(self.encoder.classes_),
                                use_label_encoder=False, eval_metric='mlogloss', random_state=SEED, n_jobs=-1)
            
            # Train on the smaller optimization training set
            clf.fit(X_train_opt, y_train_opt)
            
            # Evaluate on the dedicated validation set
            y_pred = clf.predict(X_val_opt)
            score = f1_score(y_val_opt, y_pred, average='weighted')
            
            # Hyperopt minimizes the loss, so we return the negative of the F1 score
            return {'loss': -score, 'status': STATUS_OK}

        trials = Trials()
        print("    Running Bayesian Optimization (Hyperopt)...")
        best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials, show_progressbar=False)
        best['n_estimators'] = int(best['n_estimators'])
        best['max_depth'] = int(best['max_depth'])
        print(f"    Best parameters found: {best}")

        # Train the final model on the ENTIRE balanced training set with the best parameters
        self.model = XGBClassifier(**best, objective='multi:softmax', num_class=len(self.encoder.classes_),
                                   use_label_encoder=False, eval_metric='mlogloss', random_state=SEED, n_jobs=-1)
        self.model.fit(self.X_train_bal, self.y_train_bal)
        print("    Final model trained successfully.")

    def _evaluate_and_explain(self):
        """Evaluates the model and generates SHAP explanations."""
        print("--> STAGE 6: Model Evaluation and SHAP Explainability...")
        y_pred = self.model.predict(self.X_test_scaled)
        y_pred_labels = self.encoder.inverse_transform(y_pred)
        y_true_labels = self.encoder.inverse_transform(self.y_test)

        print("\n📊 Final Model Classification Report:\n")
        print(classification_report(y_true_labels, y_pred_labels, zero_division=0))
        
        cm = confusion_matrix(y_true_labels, y_pred_labels, labels=self.encoder.classes_)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='viridis', xticklabels=self.encoder.classes_, yticklabels=self.encoder.classes_)
        plt.title("Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.show()

        print("    Generating SHAP plots...")
        explainer = shap.TreeExplainer(self.model)
        # Use the full balanced training set for SHAP values for a complete picture
        shap_values = explainer.shap_values(self.X_train_bal)
        
        X_train_bal_df = pd.DataFrame(self.X_train_bal, columns=self.feature_names)
        
        plt.figure()
        shap.summary_plot(shap_values, X_train_bal_df, plot_type="bar", show=True)
        plt.figure()
        shap.summary_plot(shap_values, X_train_bal_df, show=True)

    def save_artifacts(self, model_path='environmental_state_model.joblib', scaler_path='state_scaler.joblib', encoder_path='state_encoder.joblib'):
        """Saves the trained model, scaler, and encoder."""
        print("--> STAGE 7: Saving Model Artifacts...")
        joblib.dump(self.model, model_path)
        joblib.dump(self.scaler, scaler_path)
        joblib.dump(self.encoder, encoder_path)
        print(f"    Model saved to {model_path}")
        print(f"    Scaler saved to {scaler_path}")
        print(f"    Encoder saved to {encoder_path}")

    def run_pipeline(self):
        """Executes the full pipeline from data loading to saving artifacts."""
        self._load_and_clean_data()
        self._engineer_features_advanced()
        self._define_environmental_state()
        self._prepare_for_modeling()
        self._tune_and_train_model()
        self._evaluate_and_explain()
        self.save_artifacts()
        print("\n✅✅✅ Full pipeline executed successfully! ✅✅✅")

# ------------------------------------------------------------------------------
# SECTION B: EXECUTION
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    try:
        assessor = MicrobialRiskAssessor(file_path='/kaggle/input/iot-indoor-air-quality/IoT_Indoor_Air_Quality_Dataset.csv')
        assessor.run_pipeline()
    except FileNotFoundError:
        print("\n\n---")
        print("FATAL ERROR: 'IoT_Indoor_Air_Quality_Dataset.csv' not found.")
        print("Please make sure the dataset file is uploaded and available in the same directory.")
        print("---")
    except Exception as e:
        print(f"\n\n--- AN UNEXPECTED ERROR OCCURRED ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        print("--------------------------------------")


# V2

In [ ]:
# ------------------------------------------------------------------------------
# SECTION A: SETUP AND ENVIRONMENT
# ------------------------------------------------------------------------------
# ✅ Install compatible versions for stability
# !pip install --quiet numpy pandas scikit-learn imbalanced-learn xgboost shap hyperopt seaborn

# ✅ Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shap
import warnings

warnings.filterwarnings("ignore")

# ✅ Scikit-learn, Imblearn, XGBoost, Hyperopt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# ✅ Reproducibility
SEED = 42
np.random.seed(SEED)

class MicrobialRiskAssessor:
    """
    An advanced pipeline for assessing indoor environmental states conducive to
    microbial risk, acting as an early warning system.
    """
    def __init__(self, file_path, roll_window=24, rh_threshold=80.0):
        self.file_path = file_path
        self.roll_window = roll_window
        self.rh_threshold = rh_threshold # Humidity % for "wet" conditions
        self.df = None
        self.X = None
        self.y = None
        self.model = None
        self.scaler = None
        self.encoder = None
        self.feature_names = None
        self.explainer = None
        print("✅ MicrobialRiskAssessor initialized.")

    def _load_and_clean_data(self):
        """Loads and cleans the initial dataset."""
        print("--> STAGE 1: Loading and Cleaning Data...")
        self.df = pd.read_csv(self.file_path)
        
        self.df.rename(columns=lambda c: 
                       c.replace('(?C)', '').replace('(%)', '').replace('(ppm)', '')
                       .replace('(?g/m?)', '').replace('(ppb)', '').replace('(lux)', '')
                       .strip().replace(' ', '_'), 
                       inplace=True)
        
        self.df['Timestamp'] = pd.to_datetime(self.df['Timestamp'], dayfirst=True)
        self.df.set_index('Timestamp', inplace=True)
        self.df.sort_index(inplace=True)
        self.df.ffill(inplace=True)
        print("    Data loaded and cleaned.")

    def _engineer_features_advanced(self):
        """Generates advanced features including volatility, lag, and anomaly scores."""
        print("--> STAGE 2: Advanced Feature Engineering...")
        self.df['Hum_24_std'] = self.df['Humidity'].rolling(self.roll_window, min_periods=1).std()
        self.df['CO2_24_std'] = self.df['CO2'].rolling(self.roll_window, min_periods=1).std()

        is_wet = (self.df['Humidity'] > self.rh_threshold).astype(int)
        self.df['Time_of_Wetness_72h'] = is_wet.rolling(window=self.roll_window * 3, min_periods=1).sum()
        
        print("    Running Isolation Forest for anomaly detection...")
        numeric_cols = self.df.select_dtypes(include=np.number).columns
        if_model = IsolationForest(contamination=0.01, random_state=SEED)
        if_model.fit(self.df[numeric_cols])
        self.df['anomaly_score'] = if_model.decision_function(self.df[numeric_cols])
        
        self.df.ffill(inplace=True)
        print("    Advanced features created.")

    def _define_environmental_state(self):
        """Defines the target variable based on a hierarchy of scientifically-backed states."""
        print("--> STAGE 3: Defining Environmental State (Target Variable)...")

        def get_state(row):
            if row['Time_of_Wetness_72h'] > 4: return 'Sustained_Dampness'
            elif row['CO2'] > 1000: return 'Poor_Ventilation'
            elif row['Humidity'] > 60: return 'Elevated_Humidity'
            else: return 'Optimal'

        self.df['Environmental_State'] = self.df.apply(get_state, axis=1)
        print("    Environmental states defined.")
        print("\nEnvironmental State Distribution:")
        print(self.df['Environmental_State'].value_counts())

    def _prepare_for_modeling(self):
        """Prepares final X, y dataframes, scales, and balances the data."""
        print("--> STAGE 4: Preparing Data for Final Model...")
        self.df.dropna(inplace=True)
        self.y = self.df['Environmental_State']
        self.X = self.df.drop(columns=['Environmental_State'])
        self.X = pd.get_dummies(self.X, columns=['Ventilation_Status'], drop_first=True)
        self.feature_names = self.X.columns.tolist()

        self.encoder = LabelEncoder()
        y_encoded = self.encoder.fit_transform(self.y)

        if len(self.encoder.classes_) < 2:
            raise ValueError("The target variable has less than 2 classes. Cannot proceed.")

        split_idx = int(len(self.X) * 0.8)
        X_train, self.X_test = self.X.iloc[:split_idx], self.X.iloc[split_idx:]
        y_train, self.y_test = y_encoded[:split_idx], y_encoded[split_idx:]

        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        self.X_test_scaled = self.scaler.transform(self.X_test)

        sm = SMOTE(random_state=SEED)
        self.X_train_bal, self.y_train_bal = sm.fit_resample(X_train_scaled, y_train)
        print(f"    Data prepared. Training set balanced with SMOTE: {self.X_train_bal.shape}")

    def _tune_and_train_model(self):
        """Uses Hyperopt to find the best XGBoost parameters and trains the final model."""
        print("--> STAGE 5: Hyperparameter Tuning and Training...")
        
        X_train_opt, X_val_opt, y_train_opt, y_val_opt = train_test_split(
            self.X_train_bal, self.y_train_bal, test_size=0.2, random_state=SEED)
        
        space = {'n_estimators': hp.quniform('n_estimators', 100, 800, 50),
                 'max_depth': hp.quniform('max_depth', 4, 15, 1),
                 'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
                 'gamma': hp.uniform('gamma', 0, 0.5)}

        def objective(params):
            params['n_estimators'] = int(params['n_estimators'])
            params['max_depth'] = int(params['max_depth'])
            clf = XGBClassifier(**params, objective='multi:softmax', num_class=len(self.encoder.classes_),
                                use_label_encoder=False, eval_metric='mlogloss', random_state=SEED, n_jobs=-1)
            clf.fit(X_train_opt, y_train_opt)
            y_pred = clf.predict(X_val_opt)
            score = f1_score(y_val_opt, y_pred, average='weighted')
            return {'loss': -score, 'status': STATUS_OK}

        trials = Trials()
        print("    Running Bayesian Optimization (Hyperopt)...")
        best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials, show_progressbar=False)
        best['n_estimators'] = int(best['n_estimators']); best['max_depth'] = int(best['max_depth'])
        print(f"    Best parameters found: {best}")

        self.model = XGBClassifier(**best, objective='multi:softmax', num_class=len(self.encoder.classes_),
                                   use_label_encoder=False, eval_metric='mlogloss', random_state=SEED, n_jobs=-1)
        self.model.fit(self.X_train_bal, self.y_train_bal)
        print("    Final model trained successfully.")

    def _evaluate_and_explain(self):
        """Evaluates the model and generates SHAP explanations."""
        print("--> STAGE 6: Model Evaluation and SHAP Explainability...")
        y_pred = self.model.predict(self.X_test_scaled)
        y_pred_labels = self.encoder.inverse_transform(y_pred)
        y_true_labels = self.encoder.inverse_transform(self.y_test)

        print("\n📊 Final Model Classification Report:\n")
        print(classification_report(y_true_labels, y_pred_labels, zero_division=0))
        
        cm = confusion_matrix(y_true_labels, y_pred_labels, labels=self.encoder.classes_)
        plt.figure(figsize=(8, 6)); sns.heatmap(cm, annot=True, fmt='d', cmap='viridis', xticklabels=self.encoder.classes_, yticklabels=self.encoder.classes_)
        plt.title("Confusion Matrix"); plt.xlabel("Predicted"); plt.ylabel("Actual"); plt.show()

        print("    Generating SHAP plots...")
        self.explainer = shap.TreeExplainer(self.model)
        shap_values = self.explainer.shap_values(self.X_train_bal)
        X_train_bal_df = pd.DataFrame(self.X_train_bal, columns=self.feature_names)
        plt.figure(); shap.summary_plot(shap_values, X_train_bal_df, plot_type="bar", show=True)
        plt.figure(); shap.summary_plot(shap_values, X_train_bal_df, show=True)

    def save_artifacts(self, model_path='environmental_state_model.joblib', scaler_path='state_scaler.joblib', encoder_path='state_encoder.joblib'):
        """Saves the trained model, scaler, and encoder."""
        print("--> STAGE 7: Saving Model Artifacts...")
        joblib.dump(self.model, model_path)
        joblib.dump(self.scaler, scaler_path)
        joblib.dump(self.encoder, encoder_path)
        joblib.dump(self.feature_names, 'feature_names.joblib')
        print(f"    Model, Scaler, Encoder, and Feature List saved.")

    def run_pipeline(self):
        """Executes the full pipeline from data loading to saving artifacts."""
        self._load_and_clean_data()
        self._engineer_features_advanced()
        self._define_environmental_state()
        self._prepare_for_modeling()
        self._tune_and_train_model()
        self._evaluate_and_explain()
        self.save_artifacts()
        print("\n✅✅✅ Full pipeline executed successfully! ✅✅✅")

    def predict_and_explain_single_instance(self, input_data,
                                             model_path='environmental_state_model.joblib', 
                                             scaler_path='state_scaler.joblib', 
                                             encoder_path='state_encoder.joblib',
                                             features_path='feature_names.joblib'):
        """
        Loads the trained model and artifacts to predict the environmental state
        for a single new data point and explain the prediction.
        """
        print("\n--> NEW PREDICTION: Loading artifacts for single instance test...")
        try:
            model = joblib.load(model_path)
            scaler = joblib.load(scaler_path)
            encoder = joblib.load(encoder_path)
            feature_names = joblib.load(features_path)
        except FileNotFoundError as e:
            print(f"    ERROR: Could not load a required artifact: {e}")
            print("    Please run the full training pipeline first.")
            return

        print("    Artifacts loaded successfully.")
        
        # Create a DataFrame from the input dictionary
        input_df = pd.DataFrame([input_data])
        
        # One-hot encode ventilation status
        input_df['Ventilation_Status_Open'] = 1 if input_data.get('Ventilation_Status') == 'Open' else 0
        if 'Ventilation_Status' in input_df.columns:
            input_df = input_df.drop(columns=['Ventilation_Status'])
            
        # Ensure all required feature columns exist, filling missing ones with 0
        for col in feature_names:
            if col not in input_df.columns:
                input_df[col] = 0
        
        # Reorder columns to match the training data
        input_df = input_df[feature_names]
        
        # Scale the data using the loaded scaler
        input_scaled = scaler.transform(input_df)
        
        # Make the prediction
        prediction_encoded = model.predict(input_scaled)[0]
        prediction_proba = model.predict_proba(input_scaled)[0]
        prediction_label = encoder.inverse_transform([prediction_encoded])[0]
        
        print("\n--- PREDICTION RESULT ---")
        print(f"Predicted Environmental State: {prediction_label.upper()}")
        print("\nConfidence Scores:")
        for i, class_name in enumerate(encoder.classes_):
            print(f"  - {class_name}: {prediction_proba[i]*100:.2f}%")
            
        # --- SHAP Explanation ---
        print("\n--- PREDICTION EXPLANATION ---")
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(input_scaled)
        
        # Find the index for the predicted class
        class_index = list(encoder.classes_).index(prediction_label)
        
        print(f"The plot below shows the factors that pushed the model's prediction towards '{prediction_label}'.")
        print("Features in RED push the prediction HIGHER (towards this outcome).")
        print("Features in BLUE push the prediction LOWER (away from this outcome).")
        
        shap.initjs()
        display(shap.force_plot(explainer.expected_value[class_index], 
                                shap_values[class_index], 
                                pd.DataFrame(input_scaled, columns=feature_names), 
                                matplotlib=True))

# ------------------------------------------------------------------------------
# SECTION B: EXECUTION & TESTING
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    try:
        assessor = MicrobialRiskAssessor(file_path='/kaggle/input/iot-indoor-air-quality/IoT_Indoor_Air_Quality_Dataset.csv')
        # Step 1: Run the full training pipeline first to create the model artifacts
        assessor.run_pipeline()

        # Step 2: Now, test the trained model with new, hypothetical scenarios
        print("\n\n===================================================================")
        print("      RUNNING PREDICTION & EXPLANATION ON TEST SCENARIOS")
        print("===================================================================")

        # SCENARIO 1: Optimal Conditions
        optimal_conditions = {
            'Temperature': 21.0, 'Humidity': 50.0, 'CO2': 600, 'PM2.5': 10, 'PM10': 15,
            'TVOC': 150, 'CO': 5, 'Light_Intensity': 500, 'Motion_Detected': 1,
            'Occupancy_Count': 2, 'Ventilation_Status': 'Open',
            'Hum_24_std': 2.5, 'CO2_24_std': 50, 'Time_of_Wetness_72h': 0, 'anomaly_score': 0.1
        }
        assessor.predict_and_explain_single_instance(optimal_conditions)

        # SCENARIO 2: Poor Ventilation Scenario
        poor_ventilation = {
            'Temperature': 23.0, 'Humidity': 58.0, 'CO2': 1500, 'PM2.5': 25, 'PM10': 30,
            'TVOC': 400, 'CO': 8, 'Light_Intensity': 200, 'Motion_Detected': 1,
            'Occupancy_Count': 4, 'Ventilation_Status': 'Closed',
            'Hum_24_std': 3.0, 'CO2_24_std': 200, 'Time_of_Wetness_72h': 0, 'anomaly_score': 0.0
        }
        assessor.predict_and_explain_single_instance(poor_ventilation)
        
        # SCENARIO 3: High Humidity Scenario
        high_humidity = {
            'Temperature': 22.0, 'Humidity': 75.0, 'CO2': 800, 'PM2.5': 15, 'PM10': 20,
            'TVOC': 250, 'CO': 6, 'Light_Intensity': 100, 'Motion_Detected': 0,
            'Occupancy_Count': 1, 'Ventilation_Status': 'Closed',
            'Hum_24_std': 5.0, 'CO2_24_std': 70, 'Time_of_Wetness_72h': 2, 'anomaly_score': -0.1
        }
        assessor.predict_and_explain_single_instance(high_humidity)

    except FileNotFoundError:
        print("\n\n---")
        print("FATAL ERROR: 'IoT_Indoor_Air_Quality_Dataset.csv' not found.")
        print("Please make sure the dataset file is uploaded and available in the same directory.")
        print("---")
    except Exception as e:
        print(f"\n\n--- AN UNEXPECTED ERROR OCCURRED ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        print("--------------------------------------")


# V3

In [ ]:
# ==============================================================================
# PROJECT: ADVANCED INDOOR MICROBIAL RISK ASSESSMENT SYSTEM (8-WEEK PROJECT SCOPE)
# VERSION: 8.1 (GPU Acceleration Enabled)
#
# KEY UPGRADES:
# 1. TRULY HOLISTIC DIAGNOSTICS: This definitive version assesses ALL relevant
#    parameters (Temp, Humidity, CO2, PM2.5, PM10, TVOC, CO, Light) against
#    multi-level scientific ranges simultaneously.
# 2. ALL POSSIBLE COMBINATIONS: The target state is now a dynamic combination of
#    every identified issue, creating a powerful and precise diagnostic label.
# 3. EXPLICIT SCIENTIFIC GROUNDING: Every threshold is commented with its source
#    (ASHRAE, WHO, EPA), ensuring the model's logic is transparent.
# 4. GPU ACCELERATION: Added the 'tree_method':'gpu_hist' parameter to leverage
#    NVIDIA GPUs for significantly faster model training.
# ==============================================================================

# ------------------------------------------------------------------------------
# SECTION A: SETUP AND ENVIRONMENT
# ------------------------------------------------------------------------------
# ✅ Install compatible versions for stability
!pip install --quiet numpy pandas scikit-learn imbalanced-learn xgboost shap hyperopt seaborn

# ✅ Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shap
import warnings

warnings.filterwarnings("ignore")

# ✅ Scikit-learn, Imblearn, XGBoost, Hyperopt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# ✅ Reproducibility
SEED = 42
np.random.seed(SEED)

class EnvironmentalDiagnosticEngine:
    """
    An advanced pipeline that trains a model to diagnose a wide range of
    concurrent indoor environmental issues using multi-level, range-based
    scientific standards.
    """
    def __init__(self, file_path, roll_window=24, rh_threshold=80.0):
        self.file_path = file_path
        self.roll_window = roll_window
        self.rh_threshold = rh_threshold
        self.df = None
        self.X = None
        self.y = None
        self.model = None
        self.scaler = None
        self.encoder = None
        self.feature_names = None
        self.explainer = None
        print("✅ EnvironmentalDiagnosticEngine initialized.")

    def _load_and_clean_data(self):
        """Loads and cleans the initial dataset."""
        print("--> STAGE 1: Loading and Cleaning Data...")
        self.df = pd.read_csv(self.file_path)
        
        self.df.rename(columns=lambda c: 
                       c.replace('(?C)', '').replace('(%)', '').replace('(ppm)', '')
                       .replace('(?g/m?)', '').replace('(ppb)', '').replace('(lux)', '')
                       .strip().replace(' ', '_'), 
                       inplace=True)
        
        self.df['Timestamp'] = pd.to_datetime(self.df['Timestamp'], dayfirst=True)
        self.df.set_index('Timestamp', inplace=True)
        self.df.sort_index(inplace=True)
        self.df.ffill(inplace=True)
        print("    Data loaded and cleaned.")

    def _engineer_features_advanced(self):
        """Generates advanced features that will be used as clues by the model."""
        print("--> STAGE 2: Advanced Feature Engineering...")
        self.df['Hum_24_std'] = self.df['Humidity'].rolling(self.roll_window, min_periods=1).std()
        self.df['CO2_24_std'] = self.df['CO2'].rolling(self.roll_window, min_periods=1).std()

        is_wet = (self.df['Humidity'] > self.rh_threshold).astype(int)
        self.df['Time_of_Wetness_72h'] = is_wet.rolling(window=self.roll_window * 3, min_periods=1).sum()
        
        print("    Running Isolation Forest for anomaly detection...")
        numeric_cols = self.df.select_dtypes(include=np.number).columns
        if_model = IsolationForest(contamination=0.01, random_state=SEED)
        if_model.fit(self.df[numeric_cols])
        self.df['anomaly_score'] = if_model.decision_function(self.df[numeric_cols])
        
        self.df.ffill(inplace=True)
        print("    Advanced features created.")

    def _define_holistic_environmental_state(self):
        """
        Defines a holistic target variable by checking all relevant parameters against
        multi-level, range-based scientific standards. This is the core of the expert system.
        """
        print("--> STAGE 3: Defining Holistic Environmental State (Target Variable)...")

        def get_holistic_state(row):
            issues = []
            
            # --- Temperature (Based on ASHRAE 55 Thermal Comfort Standard) ---
            temp = row['Temperature']
            if temp > 29: issues.append('Temp-Hot')
            elif temp > 26: issues.append('Temp-Warm')
            elif temp < 18: issues.append('Temp-Cold')
            elif temp < 20: issues.append('Temp-Cool')

            # --- Humidity (Based on ASHRAE 62.1 Ventilation and IAQ Standard) ---
            hum = row['Humidity']
            if hum > 70: issues.append('Humidity-High')
            elif hum > 60: issues.append('Humidity-Elevated')
            elif hum < 30: issues.append('Humidity-Low')

            # --- CO2 (Based on ASHRAE 62.1 as a proxy for ventilation effectiveness) ---
            co2 = row['CO2']
            if co2 > 1500: issues.append('CO2-Very_Poor_Ventilation')
            elif co2 > 1000: issues.append('CO2-Poor_Ventilation')
            
            # --- PM2.5 (Fine Particulates - Based on WHO/EPA Air Quality Index) ---
            pm25 = row['PM2.5']
            if pm25 > 55.5: issues.append('PM2.5-Very_Unhealthy')
            elif pm25 > 35.5: issues.append('PM2.5-Unhealthy')
            elif pm25 > 12: issues.append('PM2.5-Moderate')

            # --- PM10 (Coarse Particulates - Based on WHO/EPA Air Quality Index) ---
            pm10 = row['PM10']
            if pm10 > 155: issues.append('PM10-Unhealthy')
            elif pm10 > 55: issues.append('PM10-Moderate')

            # --- TVOC (Total Volatile Organic Compounds - Common building science guidelines) ---
            tvoc = row['TVOC']
            if tvoc > 2000: issues.append('TVOC-High')
            elif tvoc > 600: issues.append('TVOC-Elevated')
            
            # --- CO (Carbon Monoxide - Based on WHO/EPA exposure limits) ---
            co = row['CO']
            if co > 15: issues.append('CO-High') # Short-term high exposure concern
            elif co > 9: issues.append('CO-Elevated') # Concern over 8-hour exposure
            
            # --- Light (Based on IES - Illuminating Engineering Society recommendations) ---
            light = row['Light_Intensity']
            if light > 2000: issues.append('Light-High_Glare') # Potential for glare
            elif light < 150 and pd.to_datetime(row.name).hour in range(8, 18): # Low light during daytime
                issues.append('Light-Low')

            # --- Sustained Dampness (Microbial growth risk factor) ---
            if row['Time_of_Wetness_72h'] > 4:
                # Add this only if a humidity issue isn't already flagged to avoid redundancy
                if 'Humidity-High' not in issues and 'Humidity-Elevated' not in issues:
                    issues.append('Sustained_Dampness')

            if not issues:
                return 'Optimal'
            else:
                # Sort for consistency and join all identified issues into a single diagnostic label
                return '_'.join(sorted(issues))

        self.df['Holistic_State'] = self.df.apply(get_holistic_state, axis=1)
        print("    Holistic environmental states defined.")
        print("\nHolistic State Distribution (Top 20):")
        print(self.df['Holistic_State'].value_counts().head(20))

    def _prepare_for_modeling(self):
        """Prepares final X, y dataframes, scales, and balances the data."""
        print("--> STAGE 4: Preparing Data for Final Model...")
        self.df.dropna(inplace=True)
        self.y = self.df['Holistic_State']
        self.X = self.df.drop(columns=['Holistic_State'])
        self.X = pd.get_dummies(self.X, columns=['Ventilation_Status'], drop_first=True)
        self.feature_names = self.X.columns.tolist()

        self.encoder = LabelEncoder()
        y_encoded = self.encoder.fit_transform(self.y)

        if len(self.encoder.classes_) < 2:
            raise ValueError("Target variable has less than 2 classes. Cannot proceed.")

        split_idx = int(len(self.X) * 0.8)
        X_train, self.X_test = self.X.iloc[:split_idx], self.X.iloc[split_idx:]
        y_train, self.y_test = y_encoded[:split_idx], y_encoded[split_idx:]

        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        self.X_test_scaled = self.scaler.transform(self.X_test)

        unique, counts = np.unique(y_train, return_counts=True)
        min_samples = counts.min()
        k_neighbors = max(1, min_samples - 1)

        print(f"    Balancing training set with SMOTE (k_neighbors set to {k_neighbors})...")
        sm = SMOTE(random_state=SEED, k_neighbors=k_neighbors)
        self.X_train_bal, self.y_train_bal = sm.fit_resample(X_train_scaled, y_train)
        print(f"    Data prepared. Training set balanced. Shape: {self.X_train_bal.shape}")

    def _tune_and_train_model(self):
        """Uses Hyperopt to find the best XGBoost parameters and trains the final model."""
        print("--> STAGE 5: Hyperparameter Tuning and Training...")
        
        X_train_opt, X_val_opt, y_train_opt, y_val_opt = train_test_split(
            self.X_train_bal, self.y_train_bal, test_size=0.2, random_state=SEED)
        
        space = {'n_estimators': hp.quniform('n_estimators', 100, 800, 50),
                 'max_depth': hp.quniform('max_depth', 4, 15, 1),
                 'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
                 'gamma': hp.uniform('gamma', 0, 0.5)}

        def objective(params):
            params['n_estimators'] = int(params['n_estimators']); params['max_depth'] = int(params['max_depth'])
            clf = XGBClassifier(**params, objective='multi:softmax', num_class=len(self.encoder.classes_),
                                use_label_encoder=False, eval_metric='mlogloss', random_state=SEED,
                                # --- GPU ACCELERATION ENABLED ---
                                tree_method='gpu_hist' 
                               )
            clf.fit(X_train_opt, y_train_opt)
            y_pred = clf.predict(X_val_opt)
            score = f1_score(y_val_opt, y_pred, average='weighted')
            return {'loss': -score, 'status': STATUS_OK}

        trials = Trials()
        print("    Running Bayesian Optimization (Hyperopt)... (This will be much faster with a GPU)")
        best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials, show_progressbar=False)
        best['n_estimators'] = int(best['n_estimators']); best['max_depth'] = int(best['max_depth'])
        print(f"    Best parameters found: {best}")

        # Train the final model with the best parameters, also on GPU
        self.model = XGBClassifier(**best, objective='multi:softmax', num_class=len(self.encoder.classes_),
                                   use_label_encoder=False, eval_metric='mlogloss', random_state=SEED,
                                   # --- GPU ACCELERATION ENABLED ---
                                   tree_method='gpu_hist'
                                  )
        self.model.fit(self.X_train_bal, self.y_train_bal)
        print("    Final model trained successfully.")

    def _evaluate_and_explain(self):
        """Evaluates the model and generates SHAP explanations."""
        print("--> STAGE 6: Model Evaluation and SHAP Explainability...")
        y_pred = self.model.predict(self.X_test_scaled)
        y_pred_labels = self.encoder.inverse_transform(y_pred)
        y_true_labels = self.encoder.inverse_transform(self.y_test)

        print("\n📊 Final Model Classification Report:\n")
        print(classification_report(y_true_labels, y_pred_labels, zero_division=0))
        
        cm = confusion_matrix(y_true_labels, y_pred_labels, labels=self.encoder.classes_)
        plt.figure(figsize=(16, 14)); sns.heatmap(cm, annot=True, fmt='d', cmap='viridis', xticklabels=self.encoder.classes_, yticklabels=self.encoder.classes_)
        plt.title("Confusion Matrix", fontsize=16); plt.xlabel("Predicted", fontsize=12); plt.ylabel("Actual", fontsize=12); plt.xticks(rotation=45, ha='right'); plt.yticks(rotation=0); plt.tight_layout(); plt.show()

        print("    Generating SHAP plots...")
        self.explainer = shap.TreeExplainer(self.model)
        shap_values = self.explainer.shap_values(self.X_train_bal)
        X_train_bal_df = pd.DataFrame(self.X_train_bal, columns=self.feature_names)
        plt.figure(); shap.summary_plot(shap_values, X_train_bal_df, plot_type="bar", max_display=20, show=True)
        plt.figure(); shap.summary_plot(shap_values, X_train_bal_df, max_display=20, show=True)

    def save_artifacts(self, model_path='holistic_diagnostic_model.joblib', scaler_path='holistic_scaler.joblib', encoder_path='holistic_encoder.joblib', features_path='holistic_features.joblib'):
        """Saves all artifacts required for inference."""
        print("--> STAGE 7: Saving Model Artifacts...")
        joblib.dump(self.model, model_path)
        joblib.dump(self.scaler, scaler_path)
        joblib.dump(self.encoder, encoder_path)
        joblib.dump(self.feature_names, features_path)
        print(f"    Model and artifacts for holistic engine saved.")

    def run_pipeline(self):
        """Executes the full pipeline from data loading to saving artifacts."""
        self._load_and_clean_data()
        self._engineer_features_advanced()
        self._define_holistic_environmental_state()
        self._prepare_for_modeling()
        self._tune_and_train_model()
        self._evaluate_and_explain()
        self.save_artifacts()
        print("\n✅✅✅ Full pipeline executed successfully! ✅✅✅")

# ------------------------------------------------------------------------------
# SECTION B: EXECUTION
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    try:
        engine = EnvironmentalDiagnosticEngine(file_path='/kaggle/input/iot-indoor-air-quality/IoT_Indoor_Air_Quality_Dataset.csv')
        engine.run_pipeline()
    except FileNotFoundError:
        print("\n\n--- FATAL ERROR: 'IoT_Indoor_Air_Quality_Dataset.csv' not found. ---")
    except Exception as e:
        print(f"\n\n--- AN UNEXPECTED ERROR OCCURRED ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")


# V4

In [ ]:
!pip install --quiet --no-deps --upgrade numpy pandas scikit-learn imbalanced-learn xgboost shap seaborn plotly pywavelets matplotlib joblib

In [1]:
# ==============================================================================
# PROJECT: ADVANCED INDOOR MICROBIAL RISK ASSESSMENT SYSTEM (AIMRAS)
# ------------------------------------------------------------------------------
# SECTION A: SETUP AND ENVIRONMENT
# ------------------------------------------------------------------------------

# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shap
import warnings
import logging
import json
import os
from typing import Dict, List, Tuple, Any

# Advanced visualization
import plotly.express as px
from plotly.subplots import make_subplots

# Time series analysis
import pywt  # PyWavelets for wavelet transforms

# Scikit-learn, Imblearn, XGBoost
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, f1_score,
    precision_recall_curve, roc_curve, auc
)
from sklearn.ensemble import IsolationForest, RandomForestClassifier, VotingClassifier
from sklearn.cluster import DBSCAN
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from xgboost import XGBClassifier

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("aimras_analytics.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("AIMRAS")

# Silence non-critical warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)

# ------------------------------------------------------------------------------
# SECTION B: CONFIGURATION SYSTEM
# ------------------------------------------------------------------------------

class AIMRASConfig:
    """Simplified configuration management for the AIMRAS analytics script."""
    DEFAULT_CONFIG = {
        "data": {
            "historical_file_path": "/kaggle/input/iot-indoor-air-quality/IoT_Indoor_Air_Quality_Dataset.csv",
        },
        "processing": {
            "anomaly_contamination": 0.01,
            "roll_window": 24,
        },
        "model": {
            "train_test_split": 0.8,
            "ensemble_models": ["xgboost", "random_forest"],
            "output_dir": "aimras_outputs" # Directory for saved models and graphs
        },
        "microbial_thresholds": {
            "mold": {
                "rh_sustained": 80.0,  # % RH
                "rh_duration": 48,     # hours
                "temperature_range": [5, 35],  # °C
            },
            "virus_persistence": {
                "rh_low_risk": 20, # % RH below this increases persistence
                "rh_high_risk": 80, # % RH above this increases persistence
                "co2_ventilation_proxy": 1000 # ppm, high CO2 indicates poor ventilation
            }
        }
    }

    def __init__(self, config_file=None):
        self.config = self.DEFAULT_CONFIG.copy()
        if config_file and os.path.exists(config_file):
            with open(config_file, 'r') as f:
                file_config = json.load(f)
                self._deep_update(self.config, file_config)
                logger.info(f"Loaded configuration from {config_file}")
        logger.info("Configuration initialized.")

    def _deep_update(self, d, u):
        for k, v in u.items():
            if isinstance(v, dict) and k in d and isinstance(d[k], dict):
                self._deep_update(d[k], v)
            else:
                d[k] = v

    def get(self, section, key=None):
        if key is None:
            return self.config.get(section, {})
        return self.config.get(section, {}).get(key)

# ------------------------------------------------------------------------------
# SECTION C: CORE ANALYTICS ENGINE
# ------------------------------------------------------------------------------

class EnvironmentalDiagnosticEngine:
    """
    A comprehensive engine for indoor air quality analysis, featuring advanced
    feature engineering, microbial risk assessment, and ensemble modeling.
    """
    def __init__(self, config_file=None):
        self.config = AIMRASConfig(config_file)
        self.df = None
        self.X_train, self.X_test = None, None
        self.y_train, self.y_test = None, None
        self.models = {}
        self.scaler = None
        self.encoder = None
        self.feature_names = None
        self.output_dir = self.config.get("model", "output_dir")
        os.makedirs(self.output_dir, exist_ok=True)
        logger.info(f"✅ EnvironmentalDiagnosticEngine initialized. Outputs will be saved to '{self.output_dir}/'")

    def _load_and_clean_data(self, filepath: str):
        """Loads and cleans the dataset with robust validation and processing."""
        logger.info("--> STAGE 1: Loading and Cleaning Data...")
        try:
            self.df = pd.read_csv(filepath)
            logger.info(f"Loaded {len(self.df)} rows from {filepath}")

            # Standardize column names
            self.df.rename(columns=lambda c: c.strip().replace(' ', '_'), inplace=True)
            
            # Robust datetime conversion
            self.df['Timestamp'] = pd.to_datetime(self.df['Timestamp'], errors='coerce')
            self.df = self.df.dropna(subset=['Timestamp']).set_index('Timestamp').sort_index()

            # Handle duplicates
            self.df = self.df[~self.df.index.duplicated(keep='last')]

            # Interpolate missing numeric data and forward-fill categorical
            numeric_cols = self.df.select_dtypes(include=np.number).columns
            self.df[numeric_cols] = self.df[numeric_cols].interpolate(method='time')
            self.df = self.df.ffill().bfill() # Fill any remaining gaps

            # Cap extreme outliers using the IQR method
            for col in numeric_cols:
                Q1 = self.df[col].quantile(0.25)
                Q3 = self.df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound, upper_bound = Q1 - 2.5 * IQR, Q3 + 2.5 * IQR
                outliers_count = ((self.df[col] < lower_bound) | (self.df[col] > upper_bound)).sum()
                if outliers_count > 0:
                    logger.info(f"Capping {outliers_count} outliers in '{col}'.")
                    self.df[col] = self.df[col].clip(lower_bound, upper_bound)

            logger.info(f"Data cleaned successfully. Final shape: {self.df.shape}")

        except Exception as e:
            logger.error(f"Error in data loading/cleaning: {e}")
            raise

    def _engineer_features(self):
        """Generates advanced features for temporal patterns and microbial risk."""
        logger.info("--> STAGE 2: Advanced Feature Engineering...")
        
        # Time-based features
        self.df['hour'] = self.df.index.hour
        self.df['day_of_week'] = self.df.index.dayofweek
        self.df['is_weekend'] = self.df['day_of_week'].isin([5, 6]).astype(int)
        
        # Cyclical encoding of time for models to understand time-of-day/week
        self.df['hour_sin'] = np.sin(2 * np.pi * self.df['hour'] / 24)
        self.df['hour_cos'] = np.cos(2 * np.pi * self.df['hour'] / 24)

        # Rolling statistics for key variables
        for col in ['Temperature', 'Humidity', 'CO2', 'PM2.5']:
            if col in self.df.columns:
                self.df[f'{col}_24h_mean'] = self.df[col].rolling(24, min_periods=1).mean()
                self.df[f'{col}_24h_std'] = self.df[col].rolling(24, min_periods=1).std()
                self.df[f'{col}_rate_of_change'] = self.df[col].diff().fillna(0)

        # Microbial growth risk factors
        mold_cfg = self.config.get("microbial_thresholds", "mold")
        is_wet = (self.df['Humidity'] > mold_cfg["rh_sustained"]).astype(int)
        self.df[f'time_of_wetness_{mold_cfg["rh_duration"]}h'] = is_wet.rolling(window=mold_cfg["rh_duration"], min_periods=1).sum()

        # Dew point calculation (risk of condensation)
        self.df['dew_point'] = self.df['Temperature'] - ((100 - self.df['Humidity']) / 5)

        # Anomaly detection with Isolation Forest
        numeric_cols = self.df.select_dtypes(include=np.number).columns
        if_model = IsolationForest(contamination=self.config.get("processing", "anomaly_contamination"), random_state=SEED)
        self.df['anomaly_score'] = if_model.fit_predict(self.df[numeric_cols])

        # Fill any NaNs created by rolling features
        self.df.fillna(method='bfill', inplace=True)
        self.df.fillna(0, inplace=True)
        logger.info(f"Feature engineering complete. Total features: {len(self.df.columns)}")

    def _define_holistic_environmental_state(self):
        """Defines a comprehensive target variable based on scientific thresholds."""
        logger.info("--> STAGE 3: Defining Holistic Environmental State (Target Variable)...")
        
        mold_cfg = self.config.get("microbial_thresholds", "mold")
        virus_cfg = self.config.get("microbial_thresholds", "virus_persistence")

        def get_state(row):
            issues = []
            # Thermal Comfort (ASHRAE 55)
            if row['Temperature'] > 28: issues.append('Temp-Hot')
            elif row['Temperature'] < 20: issues.append('Temp-Cold')
            
            # Humidity (ASHRAE 62.1)
            if row['Humidity'] > 70: issues.append('Humidity-High')
            elif row['Humidity'] < 30: issues.append('Humidity-Low')

            # Ventilation (CO2 as proxy)
            if 'CO2' in row and row['CO2'] > 1500: issues.append('Ventilation-Poor')
            elif 'CO2' in row and row['CO2'] > 1000: issues.append('Ventilation-Moderate')

            # Particulates (WHO/EPA)
            if 'PM2.5' in row and row['PM2.5'] > 35.5: issues.append('PM2.5-Unhealthy')
            elif 'PM2.5' in row and row['PM2.5'] > 12: issues.append('PM2.5-Moderate')

            # Mold Risk
            is_temp_conducive = mold_cfg["temperature_range"][0] <= row['Temperature'] <= mold_cfg["temperature_range"][1]
            if row[f'time_of_wetness_{mold_cfg["rh_duration"]}h'] >= mold_cfg["rh_duration"] and is_temp_conducive:
                issues.append('Risk-Mold')

            # Virus Persistence Risk
            is_rh_risky = not (virus_cfg["rh_low_risk"] < row['Humidity'] < virus_cfg["rh_high_risk"])
            is_vent_poor = 'CO2' in row and row['CO2'] > virus_cfg["co2_ventilation_proxy"]
            if is_rh_risky and is_vent_poor:
                issues.append('Risk-VirusPersistence')

            if not issues:
                return 'Optimal'
            
            # Combine top issues for a composite state
            if 'Risk-Mold' in issues: return 'High-Risk-Mold'
            if 'Risk-VirusPersistence' in issues and 'Ventilation-Poor' in issues: return 'High-Risk-Airborne'
            if len(issues) >= 3: return 'Multi-Factor-Poor'
            
            return sorted(issues)[0] # Return the most significant single issue

        self.df['Holistic_State'] = self.df.apply(get_state, axis=1)
        logger.info("Holistic environmental states defined.")
        logger.info(f"\nState Distribution:\n{self.df['Holistic_State'].value_counts(normalize=True).to_string()}")

    def _prepare_for_modeling(self):
        """Prepares data for modeling, including scaling, encoding, and resampling."""
        logger.info("--> STAGE 4: Preparing Data for Modeling...")
        
        self.y = self.df['Holistic_State']
        self.X = self.df.drop(columns=['Holistic_State'])
        self.feature_names = self.X.columns.tolist()

        # Encode target variable
        self.encoder = LabelEncoder()
        y_encoded = self.encoder.fit_transform(self.y)

        # Time-based train-test split
        train_ratio = self.config.get("model", "train_test_split")
        split_idx = int(len(self.X) * train_ratio)
        X_train, self.X_test = self.X.iloc[:split_idx], self.X.iloc[split_idx:]
        y_train, self.y_test = y_encoded[:split_idx], y_encoded[split_idx:]

        # Scale features
        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        self.X_test = self.scaler.transform(self.X_test) # Test data is scaled here

        # Handle class imbalance using SMOTE
        unique_classes, counts = np.unique(y_train, return_counts=True)
        k_neighbors = max(1, min(5, np.min(counts) - 1))
        
        if k_neighbors > 0:
            logger.info(f"Handling class imbalance with SMOTE (k_neighbors={k_neighbors})...")
            smote = SMOTE(random_state=SEED, k_neighbors=k_neighbors)
            self.X_train, self.y_train = smote.fit_resample(X_train_scaled, y_train)
            logger.info(f"Training data balanced. New shape: {self.X_train.shape}")
        else:
            logger.warning("Cannot apply SMOTE, at least one class has fewer than 2 samples. Using original data.")
            self.X_train = X_train_scaled
            self.y_train = y_train

    def _train_and_save_model(self):
        """Trains the ensemble model and saves it along with pre-processing components."""
        logger.info("--> STAGE 5: Training and Saving Model...")

        # Define individual models
        xgb = XGBClassifier(
            objective='multi:softmax', num_class=len(self.encoder.classes_),
            random_state=SEED, n_estimators=200, learning_rate=0.1, max_depth=7,
            use_label_encoder=False, eval_metric='mlogloss'
        )
        rf = RandomForestClassifier(
            n_estimators=150, max_depth=10, random_state=SEED, n_jobs=-1
        )
        
        # Create the ensemble model
        ensemble = VotingClassifier(
            estimators=[('xgboost', xgb), ('random_forest', rf)],
            voting='soft' # Use probabilities for better performance
        )
        
        logger.info("Training the soft voting ensemble classifier...")
        ensemble.fit(self.X_train, self.y_train)
        self.models['ensemble'] = ensemble
        logger.info("Ensemble model trained successfully.")

        # Save the complete model bundle
        model_bundle = {
            'model': ensemble,
            'scaler': self.scaler,
            'encoder': self.encoder,
            'feature_names': self.feature_names
        }
        model_path = os.path.join(self.output_dir, 'aimras_model_bundle.pkl')
        joblib.dump(model_bundle, model_path)
        logger.info(f"Model bundle saved to: {model_path}")


    def _evaluate_model(self):
        """Evaluates the final model and generates and saves comprehensive evaluation plots."""
        logger.info("--> STAGE 6: Model Evaluation and Visualization...")
        
        if 'ensemble' not in self.models:
            logger.error("No ensemble model found to evaluate.")
            return

        model = self.models['ensemble']
        y_pred = model.predict(self.X_test)
        y_prob = model.predict_proba(self.X_test)
        y_true_labels = self.encoder.inverse_transform(self.y_test)
        y_pred_labels = self.encoder.inverse_transform(y_pred)
        class_labels = self.encoder.classes_
        n_classes = len(class_labels)

        # 1. Classification Report
        logger.info("\n--- Classification Report ---\n")
        report_dict = classification_report(y_true_labels, y_pred_labels, target_names=class_labels, zero_division=0, output_dict=True)
        logger.info(classification_report(y_true_labels, y_pred_labels, target_names=class_labels, zero_division=0))

        # 2. Save Confusion Matrix
        plt.figure(figsize=(12, 10))
        cm = confusion_matrix(y_true_labels, y_pred_labels, labels=class_labels)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
        plt.title("Ensemble Model Confusion Matrix", fontsize=16)
        plt.xlabel("Predicted State", fontsize=12)
        plt.ylabel("Actual State", fontsize=12)
        plt.xticks(rotation=45, ha='right'); plt.yticks(rotation=0)
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, "confusion_matrix.png"))
        plt.show()

        # 3. Save SHAP Feature Importance Plot
        logger.info("Generating and saving SHAP explanations...")
        explainer = shap.TreeExplainer(self.models['ensemble'].named_estimators_['xgboost'])
        shap_values = explainer.shap_values(self.X_test)
        plt.figure()
        shap.summary_plot(shap_values, self.X_test, feature_names=self.feature_names,
                          class_names=class_labels, plot_type="bar", max_display=20, show=False)
        plt.title("SHAP Feature Importance (Top 20)", fontsize=16)
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, "shap_feature_importance.png"))
        plt.show()

        # Binarize the output for PR and ROC curves
        y_test_binarized = label_binarize(self.y_test, classes=range(n_classes))

        # 4. Save Precision-Recall Curve
        plt.figure(figsize=(12, 8))
        for i in range(n_classes):
            precision, recall, _ = precision_recall_curve(y_test_binarized[:, i], y_prob[:, i])
            plt.plot(recall, precision, lw=2, label=f'PR curve for {class_labels[i]}')
        plt.xlabel("Recall"); plt.ylabel("Precision")
        plt.title("Precision-Recall Curve for Each Class"); plt.legend(loc="best")
        plt.grid(True); plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, "precision_recall_curves.png"))
        plt.show()

        # 5. Save ROC Curve
        plt.figure(figsize=(12, 8))
        for i in range(n_classes):
            fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_prob[:, i])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, lw=2, label=f'ROC curve for {class_labels[i]} (area = {roc_auc:0.2f})')
        plt.plot([0, 1], [0, 1], 'k--', lw=2) # Dashed diagonal
        plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
        plt.title("ROC Curve for Each Class"); plt.legend(loc="lower right")
        plt.grid(True); plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, "roc_curves.png"))
        plt.show()

        # 6. Save Overall Metrics Summary Plot
        metrics = {
            'Accuracy': report_dict['accuracy'],
            'Macro Avg F1': report_dict['macro avg']['f1-score'],
            'Weighted Avg F1': report_dict['weighted avg']['f1-score']
        }
        plt.figure(figsize=(8, 6))
        sns.barplot(x=list(metrics.keys()), y=list(metrics.values()))
        plt.title('Overall Model Performance Metrics')
        plt.ylabel('Score')
        plt.ylim(0, 1)
        for index, value in enumerate(metrics.values()):
            plt.text(index, value + 0.01, f"{value:.3f}", ha='center')
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, "metrics_summary.png"))
        plt.show()


    def predict_on_new_data(self, new_data: pd.DataFrame):
        """Predicts the environmental state for new, unseen data using the saved model bundle."""
        logger.info("--> PREDICTION: Running inference on new data...")
        model_path = os.path.join(self.output_dir, 'aimras_model_bundle.pkl')
        
        try:
            bundle = joblib.load(model_path)
            model = bundle['model']
            scaler = bundle['scaler']
            encoder = bundle['encoder']
            feature_names = bundle['feature_names']
        except FileNotFoundError:
            logger.error(f"Model bundle not found at {model_path}. Please run the training pipeline first.")
            return None
        
        # Ensure new_data has the same columns as training data
        new_data_processed = new_data.reindex(columns=feature_names, fill_value=0)
        
        # Scale the data
        new_data_scaled = scaler.transform(new_data_processed)
        
        # Predict
        predictions_encoded = model.predict(new_data_scaled)
        predictions_proba = model.predict_proba(new_data_scaled)
        
        # Decode predictions to human-readable labels
        predictions_labels = encoder.inverse_transform(predictions_encoded)
        
        results = []
        for i, label in enumerate(predictions_labels):
            confidence = predictions_proba[i][predictions_encoded[i]]
            results.append({"predicted_state": label, "confidence": f"{confidence:.2%}"})
            logger.info(f"Prediction for sample {i}: {label} (Confidence: {confidence:.2%})")
            
        return results

    def run_full_pipeline(self):
        """Executes the entire analytics pipeline from start to finish."""
        logger.info("🚀 STARTING AIMRAS FULL ANALYTICS PIPELINE �")
        try:
            self._load_and_clean_data(self.config.get("data", "historical_file_path"))
            self._engineer_features()
            self._define_holistic_environmental_state()
            self._prepare_for_modeling()
            self._train_and_save_model()
            self._evaluate_model()
            logger.info("✅ AIMRAS FULL ANALYTICS PIPELINE COMPLETED SUCCESSFULLY ✅")
        except Exception as e:
            logger.critical(f"Pipeline failed with a critical error: {e}", exc_info=True)


# ------------------------------------------------------------------------------
# SECTION D: EXECUTION
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    # Initialize and run the diagnostic engine
    engine = EnvironmentalDiagnosticEngine()
    engine.run_full_pipeline()

    # --- Example of how to use the trained model for inference ---
    if os.path.exists(os.path.join(engine.output_dir, 'aimras_model_bundle.pkl')):
        print("\n" + "="*50)
        print("DEMONSTRATING PREDICTION ON NEW (TEST) DATA")
        print("="*50)
        
        # To simulate new data, we create a dummy DataFrame.
        # In a real scenario, you would load your new data here.
        sample_data = {
            'Temperature': [25.5], 'Humidity': [85.0], 'CO2': [1600], 'PM2.5': [40.0],
            'hour': [14], 'day_of_week': [2], 'is_weekend': [0], 'hour_sin': [np.sin(2*np.pi*14/24)],
            'hour_cos': [np.cos(2*np.pi*14/24)], 'Temperature_24h_mean': [24.8],
            'Temperature_24h_std': [1.2], 'Temperature_rate_of_change': [0.2],
            'Humidity_24h_mean': [82.0], 'Humidity_24h_std': [3.0], 'Humidity_rate_of_change': [1.5],
            'CO2_24h_mean': [1450], 'CO2_24h_std': [150], 'CO2_rate_of_change': [50],
            'PM2.5_24h_mean': [38.0], 'PM2.5_24h_std': [5.0], 'PM2.5_rate_of_change': [2.0],
            'time_of_wetness_48h': [48], 'dew_point': [22.5], 'anomaly_score': [1]
        }
        sample_df = pd.DataFrame(sample_data)
        
        # Make predictions using the standalone function
        predictions = engine.predict_on_new_data(sample_df)

ImportError: Numba needs NumPy 2.0 or less. Got NumPy 2.3.

## Test V1

In [ ]:
# ==============================================================================
import joblib
import pandas as pd
import numpy as np
import shap
import warnings

warnings.filterwarnings("ignore")


class ModelInferenceEngine:
    """
    A class dedicated to loading a trained model and performing inference.
    """
    def __init__(self, model_path, scaler_path, encoder_path, features_path):
        self.model = None
        self.scaler = None
        self.encoder = None
        self.feature_names = None
        self.explainer = None

        try:
            print("--> Loading trained model and artifacts...")
            self.model = joblib.load(model_path)
            self.scaler = joblib.load(scaler_path)
            self.encoder = joblib.load(encoder_path)
            self.feature_names = joblib.load(features_path)
            self.explainer = shap.TreeExplainer(self.model)
            print("    ✅ Artifacts loaded successfully.")
        except FileNotFoundError as e:
            print(f"    ❌ ERROR: Could not load a required artifact: {e}")
            print("    Please ensure the training pipeline has been run and artifacts are saved.")
            raise

    def preprocess_input(self, input_data: dict) -> pd.DataFrame:
        """
        Takes a dictionary of raw input and prepares it for the model.
        """
        # Create a DataFrame from the input dictionary
        input_df = pd.DataFrame([input_data])
        
        # One-hot encode ventilation status
        input_df['Ventilation_Status_Open'] = 1 if input_data.get('Ventilation_Status') == 'Open' else 0
        if 'Ventilation_Status' in input_df.columns:
            input_df = input_df.drop(columns=['Ventilation_Status'])
            
        # Ensure all required feature columns from training exist, filling missing ones with 0
        for col in self.feature_names:
            if col not in input_df.columns:
                input_df[col] = 0
        
        # Reorder columns to exactly match the training data order
        return input_df[self.feature_names]

    def predict(self, input_data: dict):
        """
        Performs prediction and generates a human-readable explanation.
        """
        if not all([self.model, self.scaler, self.encoder, self.explainer]):
            print("    ❌ ERROR: Inference engine not initialized correctly. Cannot predict.")
            return

        # 1. Preprocess the data
        processed_df = self.preprocess_input(input_data)
        
        # 2. Scale the data using the loaded scaler
        scaled_data = self.scaler.transform(processed_df)
        
        # 3. Make the prediction
        prediction_encoded = self.model.predict(scaled_data)[0]
        prediction_proba = self.model.predict_proba(scaled_data)[0]
        prediction_label = self.encoder.inverse_transform([prediction_encoded])[0]
        
        # 4. Print results
        print("\n--- PREDICTION RESULT ---")
        print(f"Predicted Environmental State: {prediction_label.upper()}")
        print("\nConfidence Scores:")
        for i, class_name in enumerate(self.encoder.classes_):
            print(f"  - {class_name:20s}: {prediction_proba[i]*100:5.2f}%")
            
        # 5. Generate SHAP Explanation
        print("\n--- PREDICTION EXPLANATION ---")
        shap_values = self.explainer.shap_values(scaled_data)
        class_index = list(self.encoder.classes_).index(prediction_label)
        
        print(f"The plot below shows the factors pushing the model's prediction towards '{prediction_label}'.")
        print("Features in RED have a positive impact (increase the score for this state).")
        print("Features in BLUE have a negative impact (decrease the score for this state).")
        
        shap.initjs()
        # Display the SHAP force plot for the predicted class
        display(shap.force_plot(self.explainer.expected_value[class_index], 
                                shap_values[class_index], 
                                processed_df, 
                                matplotlib=True))

def run_test_scenarios():
    """
    Defines and runs a series of test cases against the trained model.
    """
    try:
        engine = ModelInferenceEngine(
            model_path='environmental_state_model.joblib',
            scaler_path='state_scaler.joblib',
            encoder_path='state_encoder.joblib',
            features_path='feature_names.joblib'
        )
    except FileNotFoundError:
        return # Stop if artifacts are not found

    print("\n\n===================================================================")
    print("      RUNNING PREDICTION & EXPLANATION ON TEST SCENARIOS")
    print("===================================================================")

    # SCENARIO 1: Textbook "Optimal" conditions
    print("\n\n--- SCENARIO 1: Optimal Conditions ---")
    optimal_conditions = {
        'Temperature': 145.0, 'Humidity': 45.0, 'CO2': 550, 'PM2.5': 80, 'PM10': 1200,
        'TVOC': 100, 'CO': 40, 'Ventilation_Status': 'Open',
        'Time_of_Wetness_72h': 0, 'anomaly_score': 1.15 # Normal score
    }
    engine.predict(optimal_conditions)

    # SCENARIO 2: A room with many people and closed windows -> "Poor Ventilation"
    print("\n\n--- SCENARIO 2: Poor Ventilation ---")
    poor_ventilation = {
        'Temperature': 24.0, 'Humidity': 62.0, 'CO2': 1800, 'PM2.5': 30, 'PM10': 45,
        'TVOC': 500, 'CO': 9, 'Occupancy_Count': 5, 'Ventilation_Status': 'Closed',
        'Time_of_Wetness_72h': 0, 'anomaly_score': -0.05 # Slightly anomalous due to high CO2/TVOC
    }
    engine.predict(poor_ventilation)
    
    # SCENARIO 3: A humid, stuffy room -> "Elevated Humidity"
    print("\n\n--- SCENARIO 3: Elevated Humidity ---")
    high_humidity = {
        'Temperature': 22.5, 'Humidity': 72.0, 'CO2': 850, 'PM2.5': 18, 'PM10': 25,
        'TVOC': 300, 'CO': 7, 'Ventilation_Status': 'Closed',
        'Time_of_Wetness_72h': 1, # Just started being "wet"
        'anomaly_score': -0.1 
    }
    engine.predict(high_humidity)

    # SCENARIO 4: A room after a water leak, damp for a while -> "Sustained Dampness"
    print("\n\n--- SCENARIO 4: Sustained Dampness ---")
    sustained_dampness = {
        'Temperature': 23.0, 'Humidity': 85.0, 'CO2': 900, 'PM2.5': 22, 'PM10': 30,
        'TVOC': 350, 'CO': 8, 'Ventilation_Status': 'Closed',
        'Time_of_Wetness_72h': 10, # Has been wet for 10 hours in the last 3 days
        'anomaly_score': -0.2 # Highly anomalous
    }
    engine.predict(sustained_dampness)


if __name__ == '__main__':
    run_test_scenarios()


In [ ]:
# ✅ Core imports
import joblib
import pandas as pd
import numpy as np
import shap
import warnings

warnings.filterwarnings("ignore")


class ModelInferenceEngine:
    """
    A class dedicated to loading a trained model and performing inference.
    """
    def __init__(self, model_path, scaler_path, encoder_path, features_path):
        self.model = None
        self.scaler = None
        self.encoder = None
        self.feature_names = None
        self.explainer = None

        try:
            print("--> Loading trained holistic model and artifacts...")
            self.model = joblib.load(model_path)
            self.scaler = joblib.load(scaler_path)
            self.encoder = joblib.load(encoder_path)
            self.feature_names = joblib.load(features_path)
            self.explainer = shap.TreeExplainer(self.model)
            print("    ✅ Artifacts loaded successfully.")
        except FileNotFoundError as e:
            print(f"    ❌ ERROR: Could not load a required artifact: {e}")
            print("    Please ensure the training pipeline has been run and artifacts are saved.")
            raise

    def _preprocess_input(self, input_data: dict) -> pd.DataFrame:
        """
        Takes a dictionary of raw input and prepares it for the model.
        This includes creating missing features and ensuring column order.
        """
        input_df = pd.DataFrame([input_data])
        input_df['Ventilation_Status_Open'] = 1 if input_data.get('Ventilation_Status') == 'Open' else 0
        if 'Ventilation_Status' in input_df.columns:
            input_df = input_df.drop(columns=['Ventilation_Status'])
            
        for col in self.feature_names:
            if col not in input_df.columns:
                input_df[col] = 0 # Default missing features to a neutral value
        
        return input_df[self.feature_names]

    def predict(self, input_data: dict, scenario_name: str):
        """
        Performs prediction for a single data point and generates a
        human-readable explanation for the result.
        """
        print(f"\n\n--- SCENARIO: {scenario_name} ---")
        if not all([self.model, self.scaler, self.encoder, self.explainer]):
            print("    ❌ ERROR: Inference engine not initialized correctly. Cannot predict.")
            return

        processed_df = self._preprocess_input(input_data)
        scaled_data = self.scaler.transform(processed_df)
        
        prediction_encoded = self.model.predict(scaled_data)[0]
        prediction_proba = self.model.predict_proba(scaled_data)[0]
        prediction_label = self.encoder.inverse_transform([prediction_encoded])[0]
        
        print("\n--- PREDICTION RESULT ---")
        print(f"Predicted Holistic State: {prediction_label.upper()}")
        print("\nConfidence Scores (Top 5):")
        sorted_indices = np.argsort(prediction_proba)[::-1]
        for i in sorted_indices[:5]:
            if prediction_proba[i] > 0.01:
                print(f"  - {self.encoder.classes_[i]:45s}: {prediction_proba[i]*100:5.2f}%")
            
        print("\n--- PREDICTION EXPLANATION ---")
        shap_values = self.explainer.shap_values(scaled_data)
        class_index = list(self.encoder.classes_).index(prediction_label)
        
        print(f"The plot below shows the factors pushing the model's prediction towards '{prediction_label}'.")
        
        shap.initjs()
        display(shap.force_plot(self.explainer.expected_value[class_index], 
                                shap_values[class_index], 
                                processed_df, 
                                matplotlib=True))

def run_all_test_scenarios():
    """
    Defines and runs a comprehensive suite of test cases to validate the model's
    ability to diagnose different environmental conditions.
    """
    try:
        engine = ModelInferenceEngine(
            model_path='holistic_diagnostic_model.joblib',
            scaler_path='holistic_scaler.joblib',
            encoder_path='holistic_encoder.joblib',
            features_path='holistic_features.joblib'
        )
    except FileNotFoundError:
        return

    print("\n\n===================================================================")
    print("      RUNNING DIAGNOSTICS WITH HOLISTIC IEQ MODEL (v8.0)")
    print("===================================================================")

    engine.predict(
        scenario_name="Optimal Conditions",
        input_data={'Temperature': 22.0, 'Humidity': 50.0, 'CO2': 600, 'PM2.5': 10, 'PM10': 20, 'TVOC': 150, 'CO': 1, 'Light_Intensity': 400}
    )
    
    engine.predict(
        scenario_name="Too Hot Only (Office in Summer)",
        input_data={'Temperature': 30.0, 'Humidity': 55.0, 'CO2': 800, 'PM2.5': 15}
    )

    engine.predict(
        scenario_name="Poor Ventilation (Crowded Meeting Room)",
        input_data={'Temperature': 24.0, 'Humidity': 58.0, 'CO2': 1800, 'Occupancy_Count': 8}
    )

    engine.predict(
        scenario_name="High Particulates (Near a Busy Road)",
        input_data={'Temperature': 25.0, 'Humidity': 50.0, 'CO2': 700, 'PM2.5': 40, 'PM10': 80}
    )
    
    engine.predict(
        scenario_name="Composite: Too Hot & Poor Ventilation",
        input_data={'Temperature': 29.5, 'Humidity': 59.0, 'CO2': 1600, 'Occupancy_Count': 6}
    )

    engine.predict(
        scenario_name="Composite: High Humidity & High VOCs (New Paint/Furniture)",
        input_data={'Temperature': 23.0, 'Humidity': 68.0, 'CO2': 800, 'TVOC': 2500}
    )

    engine.predict(
        scenario_name="Composite: Cold, Damp, & Stuffy (Winter, No Ventilation)",
        input_data={'Temperature': 17.5, 'Humidity': 72.0, 'CO2': 1300, 'Time_of_Wetness_72h': 5}
    )

    engine.predict(
        scenario_name="Catastrophic Failure: All Issues Present (Post-flood, no power)",
        input_data={'Temperature': 28.0, 'Humidity': 85.0, 'CO2': 2500, 'PM2.5': 60, 'PM10': 100, 'TVOC': 2000, 'CO': 15, 'Time_of_Wetness_72h': 20}
    )


if __name__ == '__main__':
    run_all_test_scenarios()


# save output folder

In [ ]:
import shutil
from IPython.display import FileLink

# --- CONFIGURATION ---
OUTPUT_DIR = "/kaggle/working/"
OUTPUT_ZIP = "AIMRAS.zip"

# --- ZIP THE DIRECTORY ---
shutil.make_archive(base_name=OUTPUT_ZIP.replace(".zip", ""), format="zip", root_dir=OUTPUT_DIR)

# --- DISPLAY DOWNLOAD LINK ---
print(f"✅ Zipped {OUTPUT_DIR} to {OUTPUT_ZIP}")
display(FileLink(OUTPUT_ZIP))